# Analisis FMCG Personal Care - Gelar Rasa 2025

**Fokus:** Tren Forecasting | Innovation Radar | Cannibalization Analysis

**Data:** 1M transaksi sales | 15 produk | 10K customer reviews

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing

warnings.filterwarnings('ignore')
np.random.seed(42)

# Style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Path
DATA_DIR = Path.cwd() / "Gelar_Rasa" / "data" / "fmcg_personalcare" / "fmcg_personalcare"

print("Setup complete ✓")

## 1. Load & Prepare Data

In [ ]:
# Load data
sales = pd.read_csv(DATA_DIR / "sales.csv", parse_dates=['date'])
products = pd.read_csv(DATA_DIR / "products.csv", parse_dates=['launch_date'])
reviews = pd.read_csv(DATA_DIR / "reviews.csv", parse_dates=['date'])

# Aggregate reviews
review_stats = reviews.groupby('product_id').agg({
    'rating': ['mean', 'count'],
    'sentiment': lambda x: (x == 'Positive').mean()
}).reset_index()
review_stats.columns = ['product_id', 'avg_rating', 'review_count', 'positive_rate']

# Merge all
df = sales.merge(products, on='product_id').merge(review_stats, on='product_id', how='left')

# Fill missing
df['avg_rating'].fillna(3.0, inplace=True)
df['review_count'].fillna(0, inplace=True)
df['positive_rate'].fillna(0.5, inplace=True)

# Features
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['product_age_days'] = (df['date'] - df['launch_date']).dt.days

print(f"Data loaded: {len(df):,} transactions")
print(f"Period: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"Products: {df['product_id'].nunique()}")
print(f"Total revenue: Rp {df['revenue'].sum()/1e9:.2f}B")

## 2. Business Overview

In [ ]:
# Calculate key metrics
total_revenue = df['revenue'].sum()
total_units = df['units_sold'].sum()

# Monthly aggregation
monthly = df.groupby(df['date'].dt.to_period('M')).agg({
    'revenue': 'sum',
    'units_sold': 'sum'
})
monthly.index = monthly.index.to_timestamp()

# YoY growth
rev_2024 = df[df['year'] == 2024]['revenue'].sum()
rev_2023 = df[df['year'] == 2023]['revenue'].sum()
yoy_growth = ((rev_2024 / rev_2023) - 1) * 100 if rev_2023 > 0 else 0

# Product performance
prod_rev = df.groupby('product_name')['revenue'].sum().sort_values(ascending=False)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('FMCG Personal Care - Business Overview', fontsize=16, fontweight='bold', y=0.995)

# 1. Monthly revenue trend
ax1 = axes[0, 0]
ax1.plot(monthly.index, monthly['revenue']/1e6, marker='o', linewidth=2, color='#2E7D32', markersize=5)
ax1.fill_between(monthly.index, 0, monthly['revenue']/1e6, alpha=0.3, color='#2E7D32')

peak_month = monthly['revenue'].idxmax()
ax1.scatter(peak_month, monthly.loc[peak_month, 'revenue']/1e6, s=300, color='red', zorder=5, edgecolors='white', linewidth=3)
ax1.annotate('Peak', xy=(peak_month, monthly.loc[peak_month, 'revenue']/1e6),
            xytext=(20, 20), textcoords='offset points', fontsize=11, fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
            arrowprops=dict(arrowstyle='->', lw=2, color='red'))

ax1.set_title('Monthly Revenue Trend', fontsize=13, fontweight='bold')
ax1.set_xlabel('Month', fontsize=11)
ax1.set_ylabel('Revenue (Million Rp)', fontsize=11)
ax1.grid(alpha=0.3)

# 2. Top 10 products
ax2 = axes[0, 1]
top10 = prod_rev.head(10)[::-1]
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(top10)))
bars = ax2.barh(range(len(top10)), top10.values/1e9, color=colors, edgecolor='white', linewidth=2)
ax2.set_yticks(range(len(top10)))
ax2.set_yticklabels([name[:35] for name in top10.index], fontsize=10)

for i, v in enumerate(top10.values):
    ax2.text(v/1e9, i, f'  Rp {v/1e9:.2f}B', va='center', fontsize=9, fontweight='bold')

ax2.set_title('Top 10 Products by Revenue', fontsize=13, fontweight='bold')
ax2.set_xlabel('Revenue (Billion Rp)', fontsize=11)
ax2.grid(axis='x', alpha=0.3)

# 3. Channel distribution
ax3 = axes[1, 0]
channel_rev = df.groupby('channel')['revenue'].sum().sort_values(ascending=False)
explode = [0.1 if i == 0 else 0 for i in range(len(channel_rev))]
colors_pie = sns.color_palette('Set2', n_colors=len(channel_rev))

wedges, texts, autotexts = ax3.pie(channel_rev.values, labels=channel_rev.index, autopct='%1.1f%%',
                                     explode=explode, colors=colors_pie, startangle=90,
                                     textprops={'fontsize': 11, 'fontweight': 'bold'})
ax3.set_title('Revenue by Sales Channel', fontsize=13, fontweight='bold')

# 4. Key metrics summary
ax4 = axes[1, 1]
ax4.axis('off')

metrics_text = f"""
KEY BUSINESS METRICS

Total Revenue:        Rp {total_revenue/1e9:.2f}B
Total Units Sold:     {total_units:,}
YoY Growth (2024):    {yoy_growth:+.1f}%

Active Products:      {df['product_id'].nunique()}
Avg Rating:           {df['avg_rating'].mean():.2f}/5.0
Customer Reviews:     {reviews['review_id'].nunique():,}

Top Channel:          {channel_rev.index[0]} ({channel_rev.iloc[0]/total_revenue*100:.0f}%)
Best Product:         {prod_rev.index[0][:30]}
Peak Month:           {peak_month.strftime('%b %Y')}
"""

ax4.text(0.1, 0.5, metrics_text, fontsize=12, family='monospace',
        verticalalignment='center',
        bbox=dict(boxstyle='round,pad=1', facecolor='#E3F2FD', edgecolor='#1976D2', linewidth=3))

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print(f"Revenue: Rp {total_revenue/1e9:.2f}B | YoY: {yoy_growth:+.1f}%")
print(f"{'='*60}")

## 3. Tren Forecasting - Prediksi 3 Bulan Ke Depan

**Goal:** Estimasi revenue untuk planning inventory

In [ ]:
# Prepare weekly data
weekly = df.groupby(df['date'].dt.to_period('W'))['revenue'].sum().reset_index()
weekly['date'] = weekly['date'].dt.to_timestamp()

# Use last 52 weeks
train = weekly.tail(52).copy()

# Fit exponential smoothing
model = ExponentialSmoothing(
    train['revenue'],
    seasonal_periods=12,
    trend='add',
    seasonal='add',
    initialization_method='estimated'
).fit()

# Forecast 12 weeks (~ 3 months)
forecast = model.forecast(steps=12)
last_date = train['date'].iloc[-1]
forecast_dates = pd.date_range(start=last_date + timedelta(weeks=1), periods=12, freq='W')

# Calculate metrics
current_avg = train['revenue'].tail(4).mean()
forecast_avg = forecast.mean()
growth_rate = ((forecast_avg / current_avg) - 1) * 100
total_forecast = forecast.sum()

# Visualization
fig, ax = plt.subplots(figsize=(16, 6))

# Historical
ax.plot(train['date'], train['revenue']/1e6, label='Historical', linewidth=2.5, 
        color='#1976D2', marker='o', markersize=4)
ax.fill_between(train['date'], 0, train['revenue']/1e6, alpha=0.2, color='#1976D2')

# Forecast
ax.plot(forecast_dates, forecast.values/1e6, label='Forecast', linewidth=3, 
        color='#D32F2F', linestyle='--', marker='s', markersize=6)

# Confidence band
std = train['revenue'].std()
upper = (forecast + 1.96*std) / 1e6
lower = (forecast - 1.96*std) / 1e6
ax.fill_between(forecast_dates, lower, upper, alpha=0.2, color='#D32F2F', label='95% Confidence')

# Vertical separator
ax.axvline(x=last_date, color='black', linestyle=':', linewidth=2, alpha=0.5)

# Annotation
mid_y = ax.get_ylim()[1] * 0.8
ax.text(last_date, mid_y, '  Forecast →', fontsize=12, fontweight='bold',
       bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8))

# Summary box
summary = f"""FORECAST SUMMARY (12 Weeks)

Total Expected:  Rp {total_forecast/1e9:.2f}B
Weekly Average:  Rp {forecast_avg/1e6:.1f}M
vs Current:      {growth_rate:+.1f}%

{'📈 RECOMMENDATION: Increase inventory 15-20%' if growth_rate > 5 else '➡️  RECOMMENDATION: Maintain current levels' if growth_rate > -5 else '⚠️  RECOMMENDATION: Launch promotional campaign'}
"""

ax.text(0.02, 0.97, summary, transform=ax.transAxes, fontsize=11, 
       verticalalignment='top', family='monospace',
       bbox=dict(boxstyle='round,pad=0.8', facecolor='white', 
                edgecolor='green' if growth_rate > 0 else 'red', linewidth=3))

ax.set_title('Revenue Forecast - Next 12 Weeks', fontsize=14, fontweight='bold', pad=15)
ax.set_xlabel('Week', fontsize=12)
ax.set_ylabel('Weekly Revenue (Million Rp)', fontsize=12)
ax.legend(loc='upper right', fontsize=11, framealpha=0.95)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nForecast: Rp {total_forecast/1e9:.2f}B | Growth: {growth_rate:+.1f}%")
print(f"Action: {'INCREASE stock' if growth_rate > 5 else 'MAINTAIN level' if growth_rate > -5 else 'LAUNCH promo'}")

## 4. Innovation Radar - Portfolio Matrix

**Framework:** BCG-style analysis (Growth vs Customer Satisfaction)

In [ ]:
# Product metrics
prod_metrics = df.groupby('product_name').agg({
    'revenue': 'sum',
    'units_sold': 'sum',
    'avg_rating': 'mean',
    'positive_rate': 'mean'
}).reset_index()

# Normalize
scaler = MinMaxScaler()
prod_metrics[['rev_norm', 'units_norm', 'rating_norm', 'positive_norm']] = scaler.fit_transform(
    prod_metrics[['revenue', 'units_sold', 'avg_rating', 'positive_rate']]
)

# Scores
prod_metrics['growth_score'] = (
    prod_metrics['rev_norm'] * 0.5 + 
    prod_metrics['units_norm'] * 0.5
)
prod_metrics['satisfaction_score'] = (
    prod_metrics['rating_norm'] * 0.6 + 
    prod_metrics['positive_norm'] * 0.4
)

# Categorize
med_g = prod_metrics['growth_score'].median()
med_s = prod_metrics['satisfaction_score'].median()

def categorize(row):
    if row['growth_score'] > med_g and row['satisfaction_score'] > med_s:
        return 'STAR'
    elif row['growth_score'] > med_g:
        return 'QUESTION'
    elif row['satisfaction_score'] > med_s:
        return 'CASH COW'
    else:
        return 'DOG'

prod_metrics['category'] = prod_metrics.apply(categorize, axis=1)

# Visualization
fig, ax = plt.subplots(figsize=(14, 10))

colors_map = {
    'STAR': '#4CAF50',
    'CASH COW': '#2196F3',
    'QUESTION': '#FFC107',
    'DOG': '#F44336'
}

for cat, color in colors_map.items():
    subset = prod_metrics[prod_metrics['category'] == cat]
    ax.scatter(subset['growth_score'], subset['satisfaction_score'],
              s=subset['revenue']/1e6, alpha=0.6, color=color,
              label=f'{cat} ({len(subset)})', edgecolors='white', linewidths=2)

# Quadrant lines
ax.axhline(y=med_s, color='gray', linestyle='--', linewidth=2, alpha=0.5)
ax.axvline(x=med_g, color='gray', linestyle='--', linewidth=2, alpha=0.5)

# Labels
ax.text(0.75, 0.75, '⭐ STAR\nInvest Heavy', transform=ax.transAxes,
       fontsize=12, fontweight='bold', ha='center',
       bbox=dict(boxstyle='round', facecolor=colors_map['STAR'], alpha=0.3, linewidth=2))

ax.text(0.75, 0.25, '❓ QUESTION\nImprove Quality', transform=ax.transAxes,
       fontsize=12, fontweight='bold', ha='center',
       bbox=dict(boxstyle='round', facecolor=colors_map['QUESTION'], alpha=0.3, linewidth=2))

ax.text(0.25, 0.75, '💰 CASH COW\nMaintain', transform=ax.transAxes,
       fontsize=12, fontweight='bold', ha='center',
       bbox=dict(boxstyle='round', facecolor=colors_map['CASH COW'], alpha=0.3, linewidth=2))

ax.text(0.25, 0.25, '⚠️  DOG\nPhase Out', transform=ax.transAxes,
       fontsize=12, fontweight='bold', ha='center',
       bbox=dict(boxstyle='round', facecolor=colors_map['DOG'], alpha=0.3, linewidth=2))

ax.set_xlabel('Growth Score (Revenue + Units)', fontsize=12, fontweight='bold')
ax.set_ylabel('Customer Satisfaction (Rating + Sentiment)', fontsize=12, fontweight='bold')
ax.set_title('Innovation Radar - Product Portfolio Matrix\n(Bubble size = Revenue)', 
            fontsize=14, fontweight='bold', pad=15)
ax.legend(loc='upper left', fontsize=11, title='Category', title_fontsize=12)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(-0.05, 1.05)
ax.grid(alpha=0.2)

plt.tight_layout()
plt.show()

# Summary
print("\nPortfolio Distribution:")
for cat in ['STAR', 'CASH COW', 'QUESTION', 'DOG']:
    prods = prod_metrics[prod_metrics['category'] == cat]
    print(f"  {cat:12s}: {len(prods)} products")
    if cat in ['STAR', 'DOG'] and len(prods) > 0:
        for p in prods['product_name'].head(3):
            print(f"    - {p[:50]}")

## 5. Cannibalization Analysis

**Question:** Apakah produk baru menurunkan sales produk lama?

In [ ]:
# Monthly sales by product and type
monthly_prod = df.groupby([df['date'].dt.to_period('M'), 'type', 'product_name']).agg({
    'revenue': 'sum'
}).reset_index()
monthly_prod['month'] = monthly_prod['date'].dt.to_timestamp()

# Market share within type
monthly_prod['type_total'] = monthly_prod.groupby(['month', 'type'])['revenue'].transform('sum')
monthly_prod['share'] = monthly_prod['revenue'] / monthly_prod['type_total'] * 100

# Identify new vs old products
product_launches = df.groupby('product_name')['launch_date'].first().reset_index()
cutoff = product_launches['launch_date'].quantile(0.5)
new_products = product_launches[product_launches['launch_date'] > cutoff]['product_name'].tolist()

# Analyze cannibalization
results = []

for ptype in df['type'].unique():
    type_data = monthly_prod[monthly_prod['type'] == ptype]
    type_prods = type_data['product_name'].unique()
    
    new_in_type = [p for p in type_prods if p in new_products]
    old_in_type = [p for p in type_prods if p not in new_products]
    
    if not new_in_type or not old_in_type:
        continue
    
    new_launch = df[df['product_name'].isin(new_in_type)]['launch_date'].min()
    
    for old_prod in old_in_type:
        prod_data = type_data[type_data['product_name'] == old_prod]
        
        before = prod_data[prod_data['month'] < new_launch]['share'].mean()
        after = prod_data[prod_data['month'] >= new_launch]['share'].mean()
        
        if pd.notna(before) and pd.notna(after) and before > 0:
            change = ((after / before) - 1) * 100
            
            results.append({
                'type': ptype,
                'product': old_prod,
                'share_before': before,
                'share_after': after,
                'change_pct': change,
                'cannibalized': change < -15
            })

cannibal_df = pd.DataFrame(results).sort_values('change_pct')

# Visualization
if len(cannibal_df) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left: Change %
    top10 = cannibal_df.nsmallest(10, 'change_pct')
    colors = ['#F44336' if x < -15 else '#FFC107' for x in top10['change_pct']]
    
    bars = ax1.barh(range(len(top10)), top10['change_pct'], color=colors, edgecolor='white', linewidth=2)
    ax1.set_yticks(range(len(top10)))
    ax1.set_yticklabels([f"{row['product'][:30]}" for _, row in top10.iterrows()], fontsize=10)
    
    for i, v in enumerate(top10['change_pct']):
        ax1.text(v-1, i, f'{v:.1f}%', va='center', ha='right', fontsize=10, fontweight='bold', color='white')
    
    ax1.axvline(x=-15, color='red', linestyle='--', linewidth=2, label='Threshold (-15%)')
    ax1.axvline(x=0, color='black', linestyle='-', linewidth=1)
    
    ax1.set_title('Top 10 Cannibalized Products', fontsize=13, fontweight='bold')
    ax1.set_xlabel('Market Share Change (%)', fontsize=11)
    ax1.legend()
    ax1.grid(axis='x', alpha=0.3)
    
    # Right: Before/After
    top5 = cannibal_df.nsmallest(5, 'change_pct')
    x = np.arange(len(top5))
    width = 0.35
    
    ax2.bar(x - width/2, top5['share_before'], width, label='Before', 
           color='#4CAF50', alpha=0.8, edgecolor='white', linewidth=2)
    ax2.bar(x + width/2, top5['share_after'], width, label='After',
           color='#F44336', alpha=0.8, edgecolor='white', linewidth=2)
    
    for i, (b, a) in enumerate(zip(top5['share_before'], top5['share_after'])):
        drop = ((a/b)-1)*100
        ax2.annotate(f'{drop:.0f}%', xy=(i, max(b, a)), xytext=(0, 5),
                   textcoords='offset points', ha='center', fontsize=10,
                   fontweight='bold', color='red',
                   arrowprops=dict(arrowstyle='->', color='red', lw=1.5))
    
    ax2.set_title('Market Share: Before vs After', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Market Share (%)', fontsize=11)
    ax2.set_xticks(x)
    ax2.set_xticklabels([p[:15] for p in top5['product']], rotation=45, ha='right', fontsize=9)
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary
    n_cannibal = cannibal_df['cannibalized'].sum()
    print(f"\nCannibalization detected: {n_cannibal} products (>{-15}% drop)")
    
    if n_cannibal > 0:
        print("\nAffected products:")
        for _, row in cannibal_df[cannibal_df['cannibalized']].iterrows():
            print(f"  - {row['product'][:40]} ({row['type']}): {row['change_pct']:.1f}%")
    else:
        print("\n✓ Portfolio well-differentiated")
else:
    print("Insufficient data for analysis")

## 6. Predictive Model - Revenue Drivers

In [ ]:
# Prepare features
features = ['units_sold', 'avg_price', 'discount_pct', 'days_since_launch',
           'avg_rating', 'positive_rate', 'quarter', 'product_age_days']

model_df = df[features + ['revenue']].dropna()
X = model_df[features]
y = model_df['revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Feature importance
importance = pd.DataFrame({
    'feature': features,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=True)

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Feature importance
colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(importance)))
bars = ax1.barh(range(len(importance)), importance['importance'], color=colors, edgecolor='white', linewidth=2)
ax1.set_yticks(range(len(importance)))
ax1.set_yticklabels(importance['feature'], fontsize=11)

for i, v in enumerate(importance['importance']):
    ax1.text(v, i, f'  {v:.3f}', va='center', fontsize=10, fontweight='bold')

ax1.set_title(f'Revenue Drivers\nR² = {r2:.3f} | MAE = Rp {mae/1000:.0f}K', fontsize=13, fontweight='bold')
ax1.set_xlabel('Importance', fontsize=11)
ax1.grid(axis='x', alpha=0.3)

# Actual vs Predicted
sample_idx = np.random.choice(len(y_test), min(1000, len(y_test)), replace=False)
ax2.scatter(y_test.iloc[sample_idx], y_pred[sample_idx], alpha=0.4, s=20, color='#2196F3')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect')

ax2.set_title(f'Prediction Quality\nR² = {r2:.3f}', fontsize=13, fontweight='bold')
ax2.set_xlabel('Actual Revenue (Rp)', fontsize=11)
ax2.set_ylabel('Predicted Revenue (Rp)', fontsize=11)
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nModel R² Score: {r2:.3f} (explains {r2*100:.1f}% variance)")
print(f"Mean Absolute Error: Rp {mae/1000:.0f}K")
print(f"\nTop 3 Drivers:")
for _, row in importance.tail(3).iloc[::-1].iterrows():
    print(f"  {row['feature']:20s}: {row['importance']:.3f}")

## 7. Executive Summary

In [ ]:
print("="*70)
print("EXECUTIVE SUMMARY - FMCG PERSONAL CARE")
print("="*70)

print("\n📊 BUSINESS METRICS")
print(f"   Revenue:        Rp {total_revenue/1e9:.2f}B")
print(f"   Units Sold:     {total_units:,}")
print(f"   YoY Growth:     {yoy_growth:+.1f}%")
print(f"   Products:       {df['product_id'].nunique()}")

print("\n🔮 FORECAST (Next 3 Months)")
print(f"   Expected:       Rp {total_forecast/1e9:.2f}B")
print(f"   Growth:         {growth_rate:+.1f}%")
print(f"   Action:         {'Increase inventory 15-20%' if growth_rate > 5 else 'Maintain levels' if growth_rate > -5 else 'Launch promo campaign'}")

print("\n⭐ INNOVATION RADAR")
star_count = len(prod_metrics[prod_metrics['category'] == 'STAR'])
dog_count = len(prod_metrics[prod_metrics['category'] == 'DOG'])
print(f"   STAR products:  {star_count} (invest heavy)")
print(f"   DOG products:   {dog_count} (phase out)")

if len(cannibal_df) > 0:
    print("\n⚠️  CANNIBALIZATION")
    n_cannibal = cannibal_df['cannibalized'].sum()
    print(f"   Affected:       {n_cannibal} products")
    print(f"   Action:         {'Differentiate positioning' if n_cannibal > 0 else 'Portfolio well-managed'}")

print("\n🎯 TOP PRIORITIES")
print(f"   1. Invest in {star_count} STAR products")
print(f"   2. {'Increase' if growth_rate > 5 else 'Maintain'} inventory for Q1 2025")
print(f"   3. Phase out {dog_count} DOG products")

print("\n" + "="*70)
print("Analysis Complete")
print("="*70)